In [1]:
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import h5py
import time
plt.style.use('ggplot')
%matplotlib inline

from scipy.interpolate import interp2d


In [181]:
Pr = 0.01
S = 2.1 #for low shear cases, take 'L2_norm'. For high shear cases, take 'last_element_unity' normalization
Nsqrd = 2
Omega = 1


In [182]:
Lx = 100.0
Lz = 100.0
nx = 256
nz = 256

In [183]:
kappa = np.sqrt(Nsqrd/Pr) #This is because the box size is measured in units of d = (nu*kappa/Nsqrd)^(1/4), see Eq. (5) in https://arxiv.org/pdf/1905.06962.pdf.
nu = kappa*Pr
d = - (2*Omega - S)
e = - Nsqrd

In [184]:
kappa

1.4509525002200232

In [185]:
nu

1.3784048752090219

In [186]:
little_r = (1+ Nsqrd/( (2*Omega - S)*2 )) * (Pr/(Pr-1))
little_r

-0.0

For low shear cases, take 'L2_norm'. 

For high shear cases, take 'last_element_unity' normalization

In [187]:
#normalization_type='energy_norm' #Options are 'energy_norm', 'last_element_unity' and 'L2_norm'
normalization_type='L2_norm'

#normalization_type='first_element_unity'
#normalization_type='second_element_unity'
#normalization_type='third_element_unity'
#normalization_type='last_element_unity'


In [188]:
unstable_kx_kz_all = np.zeros((int(nx*(nz-1)), 2), dtype=np.int64)
counting = 0

gamma_mat = np.zeros((int(nx), int(nz-1), 4), dtype=np.complex128)
eigvec_mat = np.zeros((int(nx), int(nz-1), 4, 4), dtype=np.complex128)
inv_eigvec_mat = np.zeros((int(nx), int(nz-1), 4, 4), dtype=np.complex128)

for kx_mode_no in range(-int(nx/2), int(nx/2)):
    for kz_mode_no in range(-int(nz/2), int(nz/2)):
        kx = kx_mode_no*2*np.pi/Lx
        kz = kz_mode_no*2*np.pi/Lz
        k = np.sqrt(kx**2 + kz**2)
        
        if (kx_mode_no==0 and kz_mode_no==0):
            2+2
        else:

            ########################################################
            #LX = gamma* X problem is being solved below 4 x 4 matrix EVP with state vector: [ux, uy, uz, theta]. This is done bcoz, for kx=0, there is a NaN in the matrix enrty for 3x3 matrix EVP with [uy, uz, theta].
            L11 = -nu*(k**2) 
            L12 = 2*Omega*(kz**2)/(k**2)
            L13 = 0
            L14 = (kz**2)/(k**2)

            L21 = d
            L22 = -nu*(k**2) 
            L23 = 0
            L24 = 0

            L31 = 0
            L32 = -2*Omega*kx*kz/(k**2)
            L33 = -nu*(k**2)
            L34 = -kx*kz/(k**2)

            L41 = e
            L42 = 0
            L43 = 0
            L44 =  -kappa*(k**2)

            L = np.array([[L11, L12, L13, L14], [L21, L22, L23, L24], [L31, L32, L33, L34], [L41, L42, L43, L44]])

            [gamma, eigvec] = linalg.eig(L)
            
            ######################################
            ######################################
            ######################################
            #Using kinetic energy norm to normalize the eigenmodes (Nov. 23, 2022), and also made the compressible (nonexistent) mode as the 4th column for each (kx, kz).
            
            for ii in range(0, 4, 1):
                if ii == (np.where(gamma==( -nu*(k**2) ) )[0][0]):
                    2+2 #Do nothing, because inversion of that compressible mode is inconsistent and can produce nan, so do not normalize with the factor "zero."
                else:
                    energy_norm = (np.abs(eigvec[0, ii]))**2 + (np.abs(eigvec[1, ii]))**2 + (np.abs(eigvec[2, ii]))**2  #ux^2 + uy^2 + uz^2, as the positive and negative spectrum is added, 1/2 gets cancelled with 2 Real of |u \cdot u^\ast|.
                    if energy_norm==0:
                        2+2
                    else:
                        if normalization_type=='energy_norm':
                            eigvec[:, ii] = eigvec[:, ii]/((energy_norm)**(1/2))
                        if normalization_type=='first_element_unity':
                            eigvec[:, ii] = eigvec[:, ii]/(eigvec[0, ii])
                        if normalization_type=='second_element_unity':
                            eigvec[:, ii] = eigvec[:, ii]/(eigvec[1, ii])
                        if normalization_type=='third_element_unity':
                            eigvec[:, ii] = eigvec[:, ii]/(eigvec[2, ii])
                        if normalization_type=='last_element_unity':
                            eigvec[:, ii] = eigvec[:, ii]/(eigvec[3, ii])
                        if normalization_type=='L2_norm':
                            2+2 #Do nothing as the eigeenvectors are already in L2 norm.
            
            
            ######################################
            ######################################
            ######################################
            
            
            ######################################
            #kz_mode_no=0 are zonal modes, which are already the eigenmodes. So, beta_column_vector = fourier-transformed [ux,uy,uz,th] is the eigenvector. Hence, making unity here. 
            if kz_mode_no==0:
                eigvec_mat[kx_mode_no, kz_mode_no, :, 0] =  np.array([1,0,0,0]) #div(u)=0 gives, for ky=kz=0, ux=0. Thus, ux does not appear in uz or theta equation, everything gets uncoupled, and thus standard unit vectors.
                eigvec_mat[kx_mode_no, kz_mode_no, :, 1] =  np.array([0,1,0,0])
                eigvec_mat[kx_mode_no, kz_mode_no, :, 2] =  np.array([0,0,1,0])
                eigvec_mat[kx_mode_no, kz_mode_no, :, 3] =  np.array([0,0,0,1])
                gamma_mat[kx_mode_no, kz_mode_no, 0:3]     = -nu*(k**2)
                gamma_mat[kx_mode_no, kz_mode_no, 3]       = -kappa*(k**2)

            else:
                ################################
                #####Arranging the modes as unstable, stable A, stable B, pure-viscous-mode.
                ################################

                array = [0, 1, 2, 3]

                if gamma.real.max()>0:

                    #Storing the unstable mode in 0
                    gamma_mat[kx_mode_no, kz_mode_no, 0] = gamma[np.where(gamma.real>0)[0][0]]
                    eigvec_mat[kx_mode_no, kz_mode_no, :, 0] = eigvec[:, np.where(gamma.real>0)[0][0]]
                    unstable_kx_kz_all[counting, :] = [kx_mode_no, kz_mode_no]
                    counting = counting + 1

                    #Storing the stable modes in 1--3
                    array.remove(np.where(gamma.real>0)[0][0])
                    gamma_mat[kx_mode_no, kz_mode_no, 1:] = gamma[array]
                    eigvec_mat[kx_mode_no, kz_mode_no, :, 1:] = eigvec[:, array]

                else:
                    gamma_mat[kx_mode_no, kz_mode_no, :] = gamma
                    eigvec_mat[kx_mode_no, kz_mode_no, :, :] = eigvec
                    
                    
                #######Pure-viscous mode identification and storing in the last column: Dated: Mar 4, 2023
                gamma_data_copy_1  = np.zeros(4, dtype=np.complex128)
                gamma_data_copy_2  = np.zeros(4, dtype=np.complex128)
                eigvec_data_copy_1 = np.zeros((4, 4), dtype=np.complex128)
                eigvec_data_copy_2 = np.zeros((4, 4), dtype=np.complex128)
                
                array = [0, 1, 2, 3]
                
                
                pureviscous_indx = np.argmin(np.abs( np.real(gamma_mat[kx_mode_no, kz_mode_no, :]) - (-nu*(k**2)) ))
                gamma_data_copy_1[3]    =  gamma_mat[kx_mode_no, kz_mode_no,    pureviscous_indx]
                eigvec_data_copy_1[:, 3]= eigvec_mat[kx_mode_no, kz_mode_no, :, pureviscous_indx]
                
                array.remove(pureviscous_indx)
                gamma_data_copy_2[0:3]    =  gamma_mat[kx_mode_no, kz_mode_no,    array]
                eigvec_data_copy_2[:, 0]= eigvec_mat[kx_mode_no, kz_mode_no, :, array[0]]
                eigvec_data_copy_2[:, 1]= eigvec_mat[kx_mode_no, kz_mode_no, :, array[1]]
                eigvec_data_copy_2[:, 2]= eigvec_mat[kx_mode_no, kz_mode_no, :, array[2]]
                
                gamma_mat[kx_mode_no, kz_mode_no, 0:3] = gamma_data_copy_2[0:3]
                gamma_mat[kx_mode_no, kz_mode_no,   3] = gamma_data_copy_1[  3]
                
                eigvec_mat[kx_mode_no, kz_mode_no, :, 0:3] = eigvec_data_copy_2[:, 0:3]
                eigvec_mat[kx_mode_no, kz_mode_no, :,   3] = eigvec_data_copy_1[:,   3]
                
                

            ##############
            inv_eigvec_mat[kx_mode_no, kz_mode_no, :, :] = linalg.inv(eigvec_mat[kx_mode_no, kz_mode_no, :, :])


            
    if kx_mode_no%10==0:
        print(kx_mode_no)

print("Total number of (kx, kz) unstable locations: %i " %counting)

unstable_kx_kz = unstable_kx_kz_all[0: counting, :]


-120
-110
-100
-90
-80
-70
-60
-50
-40
-30
-20
-10
0
10
20
30
40
50
60
70
80
90
100
110
120
Total number of (kx, kz) unstable locations: 110 


Single mode based QL-like transport flux is calculated in the next few cells.

In [189]:
indices_largestgamma_QL = np.where(np.max(np.real(gamma_mat[0, :, :]))==np.real(gamma_mat[0, :, :]))
indices_largestgamma_QL

(array([  3, 252]), array([0, 0]))

In [190]:
gamma_QL = gamma_mat[0, indices_largestgamma_QL[0][0]]
gamma_QL

array([ 0.12291969+0.j        , -0.13621195+0.14960232j,
       -0.13621195-0.14960232j, -0.04897552+0.j        ])

In [191]:
kz_QL = (indices_largestgamma_QL[0][0]*2*np.pi/Lz)

In [192]:
s_nu    = gamma_QL[0] + nu*kz_QL**2
s_kappa = gamma_QL[0] + kappa*kz_QL**2
s_nu

(0.1718952085748192+0j)

In [193]:
uxuy_QL = (S-2*Omega)/(2*s_nu) * (gamma_QL[0]/kz_QL)**2
uxuy_QL

(1.2369350523774922+0j)

In [194]:
uxth_QL = - Nsqrd/(2*s_kappa) * (gamma_QL[0]/kz_QL)**2
uxth_QL

(-2.437321193926687+0j)

QL flux calculatioin is now complete

In [195]:
d_parameter = (nu*kappa/Nsqrd)**(1/4)
d_parameter

0.9999999999999999

In [196]:
kmin = (2*np.pi/Lx)
kmin

def nonlinearity(ux_mm, uy_mm, uz_mm, th_mm, ux_nn, uy_nn, uz_nn, th_nn, kx_ind, kz_ind, kx_pp_ind, kz_pp_ind):
    
    u_dotgrad_ux = ux_mm*(1.0j*kx_pp_ind*kmin)*ux_nn + uz_mm*(1.0j*kz_pp_ind*kmin)*ux_nn 
    u_dotgrad_uy = ux_mm*(1.0j*kx_pp_ind*kmin)*uy_nn + uz_mm*(1.0j*kz_pp_ind*kmin)*uy_nn 
    u_dotgrad_uz = ux_mm*(1.0j*kx_pp_ind*kmin)*uz_nn + uz_mm*(1.0j*kz_pp_ind*kmin)*uz_nn 
    u_dotgrad_th = ux_mm*(1.0j*kx_pp_ind*kmin)*th_nn + uz_mm*(1.0j*kz_pp_ind*kmin)*th_nn 
    
    if (kx_ind**2 + kz_ind**2)==0:
        pressure_NL=0
    else:
        pressure_NL = ((1.0j*kx_ind*kmin)*u_dotgrad_ux + (1.0j*kz_ind*kmin)*u_dotgrad_uz)/((kx_ind**2 + kz_ind**2)*kmin**2)
    
    complete_state_NL = (-1)*np.array([u_dotgrad_ux + 1.0j*kx_ind*kmin*pressure_NL, 
                                       u_dotgrad_uy, 
                                       u_dotgrad_uz + 1.0j*kz_ind*kmin*pressure_NL, 
                                       u_dotgrad_th])
    return complete_state_NL
    

In [197]:
#Take a very large network, like -15 to +15 so that, when I compute k''=k-k' will be sensible even if k and k' are for a special case k-k'=-10-(10)=-20, 
#for which I will ahve stored the coupling coefficent. Thus, anything outside of the box with abs(k, k', or k'')=10 or so will be untrustworthy 
#but inside that abs()=10 can be trusted.  In the actual analysis, I will be using a small box of -5 to +5 in kx and -5 to +5 in kz, or so.

k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude = int(2.1*np.max(np.abs(unstable_kx_kz))) #I have now chosen 3 as it turns out any number beyond 2.00001 
#is fine and does not affect the later calculations as |beta|^2 are found within a square region with half of this presceribed number.

kx_negative_max   = -k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kz_negative_max   = -k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kx_positive_max   =  k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kz_positive_max   =  k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude

kx_p_negative_max = -k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kz_p_negative_max = -k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kx_p_positive_max =  k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude
kz_p_positive_max =  k_safe_large_twice_bigger_than_max_k_used_in_closure_mode_amplitude


In [199]:
#type_of_saturation_theory = 'all_zonal_all'           # Options are 'unstable_zonal_unstable' and 'all_zonal_all'
type_of_saturation_theory = 'unstable_zonal_unstable' # Options are 'unstable_zonal_unstable' and 'all_zonal_all'


In [200]:
zonal_kx = 1




#Choose 1 first---for low shear parameter cases that is fine. Then if needed change to 2 , then 3, ...
#This is the Fourier mode number of the zonal flow which is coupled with other eigenmodes.

In [201]:
C_coeff = np.zeros((3,3,3,(kx_positive_max-kx_negative_max+1), (kz_positive_max-kz_negative_max+1), (kx_p_positive_max-kx_p_negative_max+1), (kz_p_positive_max-kz_p_negative_max+1)), dtype=np.complex128)
#Because I have now arranged eigvec_mat such that the last fourth mode is a pure viscous mode, that has zero projection because it is compressbile, non-realizable.


C_coeff_symm = 0*C_coeff 


In [202]:
betasquared_all_in_a_mat = np.zeros((kx_positive_max+1, int(kz_positive_max/2)+1, nx), dtype=np.float64)

In [203]:
def C_coeff_intermediate_step(counter, jj, mm, nn):
    for kx_ind in range(kx_negative_max, kx_positive_max+1):
        for kx_p_ind in range(kx_p_negative_max, kx_p_positive_max+1):
            for kz_ind in range(kz_negative_max, kz_positive_max+1):
                for kz_p_ind in range(kz_p_negative_max, kz_p_positive_max+1):

                    kx_pp_ind = kx_ind - kx_p_ind
                    kz_pp_ind = kz_ind - kz_p_ind

                    if counter==0:
                        if ((kx_ind==kx_p_ind+zonal_kx) or (kx_ind==kx_p_ind-zonal_kx)) and (kz_ind==kz_p_ind+0):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if counter==1:
                        if ((kx_ind==zonal_kx+kx_pp_ind) or (kx_ind==-zonal_kx+kx_pp_ind)) and (kz_ind==0+kz_pp_ind):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if counter==2:
                        if ((zonal_kx==kx_p_ind+kx_pp_ind) or (-zonal_kx==kx_p_ind+kx_pp_ind)) and (0==kz_p_ind+kz_pp_ind):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if calc_should_proceed == True:
                        ux_mm = eigvec_mat[kx_p_ind,  kz_p_ind,  0, mm]
                        uy_mm = eigvec_mat[kx_p_ind,  kz_p_ind,  1, mm]
                        uz_mm = eigvec_mat[kx_p_ind,  kz_p_ind,  2, mm]
                        th_mm = eigvec_mat[kx_p_ind,  kz_p_ind,  3, mm]

                        ux_nn = eigvec_mat[kx_pp_ind, kz_pp_ind, 0, nn]
                        uy_nn = eigvec_mat[kx_pp_ind, kz_pp_ind, 1, nn]
                        uz_nn = eigvec_mat[kx_pp_ind, kz_pp_ind, 2, nn]
                        th_nn = eigvec_mat[kx_pp_ind, kz_pp_ind, 3, nn]

                        C_coeff[jj, mm, nn, kx_ind, kz_ind, kx_p_ind, kz_p_ind] = np.dot(inv_eigvec_mat[kx_ind, kz_ind, :, jj], 
                                                                                                nonlinearity(ux_mm, uy_mm, uz_mm, th_mm, ux_nn, uy_nn, uz_nn, th_nn, kx_ind, kz_ind, kx_pp_ind, kz_pp_ind))

        if kx_ind%20==0:
            print(r"jj: %i, mm:%i, nn:%i, kx_ind:%i" %(jj, mm, nn, kx_ind))



In [204]:
def C_coeff_symm_intermediate_step(counter, jj, mm, nn):
    for kx_ind in range(kx_negative_max, kx_positive_max+1):
        for kx_p_ind in range(kx_p_negative_max, kx_p_positive_max+1):
            for kz_ind in range(kz_negative_max, kz_positive_max+1):
                for kz_p_ind in range(kz_p_negative_max, kz_p_positive_max+1):

                    kx_pp_ind = kx_ind - kx_p_ind
                    kz_pp_ind = kz_ind - kz_p_ind


                    if counter==0:
                        if ((kx_ind==kx_p_ind+zonal_kx) or (kx_ind==kx_p_ind-zonal_kx)) and (kz_ind==kz_p_ind+0):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if counter==1:
                        if ((kx_ind==zonal_kx+kx_pp_ind) or (kx_ind==-zonal_kx+kx_pp_ind)) and (kz_ind==0+kz_pp_ind):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if counter==2:
                        if ((zonal_kx==kx_p_ind+kx_pp_ind) or (-zonal_kx==kx_p_ind+kx_pp_ind)) and (0==kz_p_ind+kz_pp_ind):
                            calc_should_proceed = True
                        else:
                            calc_should_proceed = False

                    if calc_should_proceed == True:
                        C_coeff_symm[jj, mm, nn, kx_ind, kz_ind, kx_p_ind, kz_p_ind] = ( C_coeff[jj, mm, nn, kx_ind, kz_ind, kx_p_ind,  kz_p_ind] 
                                                                                       + C_coeff[jj, nn, mm, kx_ind, kz_ind, kx_pp_ind, kz_pp_ind] )


        if kx_ind%20==0:
            print(r"jj: %i, mm:%i, nn:%i, kx_ind:%i" %(jj, mm, nn, kx_ind))



In [205]:

##################################################################################################################
##################################################################################################################
##################################################################################################################
if type_of_saturation_theory == 'unstable_zonal_unstable':
    for counter in range(0,3):
        if counter==0:
            nn = 2 #Here zonal (uz) wavenumber is k''. #jj, mm, nn are at wavenumbers k, k', and k''.
            jj, mm = 0, 0
        if counter==1:
            mm = 2 #Here zonal (uz) wavenumber is k'. #jj, mm, nn are at wavenumbers k, k', and k''.
            jj, nn = 0, 0
        if counter==2:
            jj = 2 #Here zonal (uz) wavenumber is k. #jj, mm, nn are at wavenumbers k, k', and k''.
            mm, nn = 0, 0

        C_coeff_intermediate_step(counter, jj, mm, nn)
    
    
##################################################################################################################
##################################################################################################################
##################################################################################################################
if type_of_saturation_theory == 'all_zonal_all':
    for counter in range(0,3):
        if counter==0:
            nn = 2 #Here zonal (uz) wavenumber is k''. #jj, mm, nn are at wavenumbers k, k', and k''.
            for jj in range(0,3):
                for mm in range(0,3):
                    C_coeff_intermediate_step(counter, jj, mm, nn)
    
            
        if counter==1:
            mm = 2 #Here zonal (uz) wavenumber is k'. #jj, mm, nn are at wavenumbers k, k', and k''.
            for jj in range(0,3):
                for nn in range(0,3):
                    C_coeff_intermediate_step(counter, jj, mm, nn)
                    
                    
        if counter==2:
            jj = 2 #Here zonal (uz) wavenumber is k. #jj, mm, nn are at wavenumbers k, k', and k''.
            for mm in range(0,3):
                for nn in range(0,3):
                    C_coeff_intermediate_step(counter, jj, mm, nn)

        
        

jj: 0, mm:0, nn:2, kx_ind:0
jj: 0, mm:2, nn:0, kx_ind:0
jj: 2, mm:0, nn:0, kx_ind:0


In [206]:
##############################  
#Symmetrized C should be found after C is fully filled in. Otherwise, some zeros can be added and it is only around half of symmetrizex C for some Cs. This is avoided now.


##################################################################################################################
##################################################################################################################
##################################################################################################################
if type_of_saturation_theory == 'unstable_zonal_unstable':
    for counter in range(0,3):
        if counter==0:
            nn = 2 #Here zonal (uz) wavenumber is k''. #jj, mm, nn are at wavenumbers k, k', and k''.
            jj, mm = 0, 0
        if counter==1:
            mm = 2 #Here zonal (uz) wavenumber is k'. #jj, mm, nn are at wavenumbers k, k', and k''.
            jj, nn = 0, 0
        if counter==2:
            jj = 2 #Here zonal (uz) wavenumber is k. #jj, mm, nn are at wavenumbers k, k', and k''.
            mm, nn = 0, 0

        C_coeff_symm_intermediate_step(counter, jj, mm, nn)
    
    
##################################################################################################################
##################################################################################################################
##################################################################################################################
if type_of_saturation_theory == 'all_zonal_all':
    for counter in range(0,3):
        if counter==0:
            nn = 2 #Here zonal (uz) wavenumber is k''. #jj, mm, nn are at wavenumbers k, k', and k''.
            for jj in range(0,3):
                for mm in range(0,3):
                    C_coeff_symm_intermediate_step(counter, jj, mm, nn)
    
            
        if counter==1:
            mm = 2 #Here zonal (uz) wavenumber is k'. #jj, mm, nn are at wavenumbers k, k', and k''.
            for jj in range(0,3):
                for nn in range(0,3):
                    C_coeff_symm_intermediate_step(counter, jj, mm, nn)
                    
                    
        if counter==2:
            jj = 2 #Here zonal (uz) wavenumber is k. #jj, mm, nn are at wavenumbers k, k', and k''.
            for mm in range(0,3):
                for nn in range(0,3):
                    C_coeff_symm_intermediate_step(counter, jj, mm, nn)

    

jj: 0, mm:0, nn:2, kx_ind:0
jj: 0, mm:2, nn:0, kx_ind:0
jj: 2, mm:0, nn:0, kx_ind:0


In [207]:
for counter in range(0, int(unstable_kx_kz.shape[0])):
    dominant_kx, dominant_kz = unstable_kx_kz[counter, :]


    ########################
    ########################
    #Finding the mode-amplitude |\beta|^2 for the mode (kx=\pm dominant_kx, kz=\pm dominant_kx), which has largest contribution in the eigenvector above, eigvec[46, 84].
    ########################
    ########################
    expression = 0

    kx_ind = zonal_kx
    kz_ind = 0

    
    ###########################################
    ###########################################
    ###########################################
    Eigmod_Ind = 0 #This represents the two zeros modes, or to be precise, two unstable modes interacting with a zonal mode.
    factor = 1.00
    
    #______________________________________________________________________
    kx_pp_ind  =  dominant_kx #Defined w.r.t to beta'', appearing in the closure. 
    kz_pp_ind  =  dominant_kz

    kx_p_ind = kx_ind - kx_pp_ind
    kz_p_ind = kz_ind - kz_pp_ind
    C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
    C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
    gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
    expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
    #______________________________________________________________________
    kx_pp_ind  = -dominant_kx #Defined for the hermitian conjugation mode. (kx=\pm dominant_kx, kz=\pm dominant_kx)
    kz_pp_ind  = -dominant_kz

    kx_p_ind = kx_ind - kx_pp_ind
    kz_p_ind = kz_ind - kz_pp_ind
    C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
    C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
    gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
    expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
    #______________________________________________________________________
    kx_p_ind  =  dominant_kx #Defined w.r.t to beta, appearing in the closure. 
    kz_p_ind  =  dominant_kz

    kx_pp_ind  = kx_ind - kx_p_ind
    kz_pp_ind  = kz_ind - kz_p_ind
    C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
    C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
    gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
    expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
    #______________________________________________________________________
    kx_p_ind  =  -dominant_kx
    kz_p_ind  =   dominant_kz

    kx_pp_ind  = kx_ind - kx_p_ind
    kz_pp_ind  = kz_ind - kz_p_ind
    C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
    C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
    gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
    expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
    
    
    ############################################################
    ############################################################
    ############################################################
    if type_of_saturation_theory == 'all_zonal_all': #'unstable_zonal_unstable_AND_stable_zonal_stable':
        
        ###########################################
        ###########################################
        ###########################################
        Eigmod_Ind = 1 #This represents the two 1 modes, or to be precise, two stable modes, no matter what type, 
        #I will be taking self-conjugate product to get |beta_j'|^2 and use j=1 once and then j=2 next. As long as those two modes, with asterisk 
        #and without are used they leave no phase information and I am good with that. The amplitdues, I then express, as 
        #      np.abs(np.real(\gamma_1)) |beta_1|^2 = np.abs(np.real(\gamma_2)) |beta_2|^2 = 1/2 * \gamma_0 |\beta_0|^2;
        #I am using the principle of equipartition of dissipation energy by the stable modes, prallely, unlike a serial cascade and dissipation at small scales
        #Based on PoP, Terry et al.--- Mode-space energy in gyrokinetic turbulence...
        
        factor = 1/2 * np.abs(np.real(gamma_mat[dominant_kx, dominant_kz, 0])) / np.abs(np.real(gamma_mat[dominant_kx, dominant_kz, Eigmod_Ind]))

        #______________________________________________________________________
        kx_pp_ind  =  dominant_kx #Defined w.r.t to beta'', appearing in the closure. 
        kz_pp_ind  =  dominant_kz

        kx_p_ind = kx_ind - kx_pp_ind
        kz_p_ind = kz_ind - kz_pp_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_pp_ind  = -dominant_kx #Defined for the hermitian conjugation mode. (kx=\pm dominant_kx, kz=\pm dominant_kx)
        kz_pp_ind  = -dominant_kz

        kx_p_ind = kx_ind - kx_pp_ind
        kz_p_ind = kz_ind - kz_pp_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_p_ind  =  dominant_kx #Defined w.r.t to beta, appearing in the closure. 
        kz_p_ind  =  dominant_kz

        kx_pp_ind  = kx_ind - kx_p_ind
        kz_pp_ind  = kz_ind - kz_p_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_p_ind  =  -dominant_kx
        kz_p_ind  =   dominant_kz

        kx_pp_ind  = kx_ind - kx_p_ind
        kz_pp_ind  = kz_ind - kz_p_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor



        ###########################################
        ###########################################
        ###########################################
        Eigmod_Ind = 2 #This represents the two 1 modes, or to be precise, two stable modes, no matter what type, 
        #I will be taking self-conjugate product to get |beta_j'|^2 and use j=1 once and then j=2 next. As long as those two modes, with asterisk 
        #and without are used they leave no phase information and I am good with that. The amplitdues, I then express, as 
        #      np.abs(np.real(\gamma_1)) |beta_1|^2 = np.abs(np.real(\gamma_2)) |beta_2|^2 = 1/2 * \gamma_0 |\beta_0|^2;
        #I am using the principle of equipartition of dissipation energy by the stable modes, prallely, unlike a serial cascade and dissipation at small scales
        #Based on PoP, Terry et al.--- Mode-space energy in gyrokinetic turbulence...
        
        factor = 1/2 * np.abs(np.real(gamma_mat[dominant_kx, dominant_kz, 0])) / np.abs(np.real(gamma_mat[dominant_kx, dominant_kz, Eigmod_Ind]))

        #______________________________________________________________________
        kx_pp_ind  =  dominant_kx #Defined w.r.t to beta'', appearing in the closure. 
        kz_pp_ind  =  dominant_kz

        kx_p_ind = kx_ind - kx_pp_ind
        kz_p_ind = kz_ind - kz_pp_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_pp_ind  = -dominant_kx #Defined for the hermitian conjugation mode. (kx=\pm dominant_kx, kz=\pm dominant_kx)
        kz_pp_ind  = -dominant_kz

        kx_p_ind = kx_ind - kx_pp_ind
        kz_p_ind = kz_ind - kz_pp_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_p_ind,  kz_p_ind,   kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_p_ind  =  dominant_kx #Defined w.r.t to beta, appearing in the closure. 
        kz_p_ind  =  dominant_kz

        kx_pp_ind  = kx_ind - kx_p_ind
        kz_pp_ind  = kz_ind - kz_p_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor
        #______________________________________________________________________
        kx_p_ind  =  -dominant_kx
        kz_p_ind  =   dominant_kz

        kx_pp_ind  = kx_ind - kx_p_ind
        kz_pp_ind  = kz_ind - kz_p_ind
        C_unsymm  = C_coeff[2, Eigmod_Ind, Eigmod_Ind, kx_ind,    kz_ind,     kx_p_ind,  kz_p_ind]
        C_symm    = C_coeff_symm[Eigmod_Ind, 2, Eigmod_Ind, kx_pp_ind, kz_pp_ind,  kx_ind,    kz_ind  ] 
        gamma_sum = gamma_mat[kx_p_ind, kz_p_ind, Eigmod_Ind] + gamma_mat[kx_pp_ind, kz_pp_ind, Eigmod_Ind] + np.conjugate(gamma_mat[kx_ind, kz_ind, 2])
        expression= expression + np.real(1.0j*C_unsymm*C_symm/gamma_sum)*factor




    if expression!=0:
        betasquared_all_in_a_mat[dominant_kx, dominant_kz, zonal_kx] = (-1)*np.real(gamma_mat[kx_ind, kz_ind, 2]) / expression #I have now added a minus sign to make the rhs positive, 
    #in accordance with Terry 2018 and Terry 2021 PRL.

    #eigvec_betasqrd_mat_with_amp_closure_calc = np.sqrt(betasquared_all_in_a_mat[dominant_kx, dominant_kz])*eigvec_mat[dominant_kx, dominant_kz, :, 0]
    #transport_momentum_all[dominant_kx, dominant_kz] = 2*np.real(eigvec_betasqrd_mat_with_amp_closure_calc[0]*eigvec_betasqrd_mat_with_amp_closure_calc[1]) #2 is becaus eof the spectrum is taken as u_x^\ast u_y + u_x u_y^\ast
    #transport_heat_all[dominant_kx, dominant_kz]     = 2*np.real(eigvec_betasqrd_mat_with_amp_closure_calc[0]*eigvec_betasqrd_mat_with_amp_closure_calc[3]) #2 is becaus eof the spectrum is taken as u_x^\ast u_y + u_x u_y^\ast
    if counter%100==0:
        print(dominant_kx)
        

-4
3


In [208]:
betasqrd_sorted = np.sort(betasquared_all_in_a_mat[np.abs(betasquared_all_in_a_mat)<1e50])
betasqrd_inv = 1/betasqrd_sorted[betasqrd_sorted>0]
betasqrd_inv

array([5.49738421, 4.5547065 , 0.96706859, 0.83662236])

In [209]:
EDQNM_weighted_betasqrd = 1/np.sum(betasqrd_inv)
EDQNM_weighted_betasqrd

0.08434703241974738

In [210]:

#betasquared_all_in_a_mat = np.zeros((kx_positive_max+1, int(kz_positive_max/2)+1), dtype=np.float64)
transport_momentum_all =  np.zeros((nx, nz-1), dtype=np.float64)
transport_heat_all     =  np.zeros((nx, nz-1), dtype=np.float64)

for counter in range(0, unstable_kx_kz.shape[0]):
    dominant_kx, dominant_kz = unstable_kx_kz[counter, :]

    eigvec_betasqrd_mat_with_amp_closure_calc = 1 * eigvec_mat[dominant_kx, dominant_kz, :, 0] #Unit amplitude
    
    transport_momentum_all[dominant_kx, dominant_kz] = np.real(eigvec_betasqrd_mat_with_amp_closure_calc[0]*eigvec_betasqrd_mat_with_amp_closure_calc[1]) #2 is not used in front, although that come from Hermitian symmetry, 
    #as I will add explicitly in the later cell, all +ve and -ve kx and kz modes.
    
    transport_heat_all[dominant_kx, dominant_kz]     = np.real(eigvec_betasqrd_mat_with_amp_closure_calc[0]*eigvec_betasqrd_mat_with_amp_closure_calc[3]) #2 is not used in front, although that come from Hermitian symmetry, 
    #as I will add explicitly in the later cell, all +ve and -ve kx and kz modes.
    
    if counter%100==0:
        print(dominant_kx)
    

-4
3


In [211]:
transport_momentum_all_useful = np.sort(transport_momentum_all[np.abs(transport_momentum_all)<1e50])
predicted_Reynolds_stress = np.sum(transport_momentum_all_useful)*EDQNM_weighted_betasqrd
predicted_Reynolds_stress

0.3760426105279035

In [212]:
transport_heat_all_useful = np.sort(transport_heat_all[np.abs(transport_heat_all)<1e50])
predicted_heat_flux = np.sum(transport_heat_all_useful)*EDQNM_weighted_betasqrd
predicted_heat_flux

-0.7268735646611183